In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.offline as pyo
import plotly.express as px
pyo.init_notebook_mode(connected=True)
import utils as u

In [3]:
"""
Plain old dataset:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)

Enhanced with impressions:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)
"""

'\nPlain old dataset:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n\nEnhanced with impressions:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n'

In [4]:
"""
These destinations are selected based on their query pool sizes, and how representative
they are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)

9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),
482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),
9D883ED995259EA3E715122397FE0EF9 has query pool of size 2
"""
# selected_destinations = ["9FF55", "482E9", "9D883"]

'\nThese destinations are selected based on their query pool sizes, and how representative\nthey are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)\n\n9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),\n482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),\n9D883ED995259EA3E715122397FE0EF9 has query pool of size 2\n'

In [5]:

# def extra_df_prep(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.assign(
#         destination=df.apply(
#             lambda row: row.destination[0:5],
#             axis=1,
#         )
#     )
#     global selected_destinations
#     return df[df.destination.isin(selected_destinations)]


In [6]:
# unaugmented path
path = "ray/criteo/bias_varying_epoch_size"

In [7]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [8]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [9]:

# u.plot_budget_consumption_lines(
#     path,
#     epoch_size=7,
#     by_destination=True,
#     extra_df_prep=extra_df_prep
# )

In [10]:
# u.save_data(path)

In [11]:
# augmented impressions path
path = "ray/criteo/augmented_bias_varying_epoch_size"

In [12]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [13]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [14]:

# u.plot_budget_consumption_lines(
#     path,
#     epoch_size=7,
#     by_destination=True,
#     extra_df_prep=extra_df_prep
# )

In [15]:
# augmented convesrions path
path = "ray/criteo/augmented_conversions_bias_varying_epoch_size"

In [16]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [17]:
# u.save_data(path)

In [18]:
df = pd.read_csv("../data/criteo/criteo_query_pool_augmented_conversions.csv")
df.head()

,partner_id,user_id,filter,product_category3,count,conversion_timestamp,epsilon,aggregatable_cap_value,key
0,9D9E93D1D461D7BAE47FB67EC0E01B62,757E6E6ECE2242C09222A395F5C0E1B6,NaN,95A082A08B36796D9679070A169EE4B9,5.0,1596439484,0.717316,5,139
1,9D9E93D1D461D7BAE47FB67EC0E01B62,19610A2A13924049BFA6AB692AB49DD3,NaN,009F21AE727F749C597752589DE82C9A,5.0,1596439550,0.632578,5,145
2,C13DEAF04388B219AC04416DF3DA0ED8,4CA3BDF6304103161BEFC3DEAE5D0C2A,NaN,FE31DD5AD3E857EBD4BA2C4FDF732ED4,1.0,1596439563,0.644982,5,0
3,F0FD783189F55BAFC331AD347EAE6863,F3CAE70720CC4D6F8468FCAA34F85AB3,NaN,555EE90CF6686FFF818AC7065AAB2773,3.0,1596439655,0.511686,5,1
4,5DC3ED9F13993DF6471EBE25BD88BD24,F0D52A53241B145050B5ED8C55DD4B5B,NaN,8A30BF8BD2B3E9D44D2938D42C387D2E,1.0,1596439718,0.699874,5,0


In [19]:
true_output = df.groupby(["partner_id", "key"])["count"].sum()

In [20]:
print(true_output.shape, true_output[true_output > 0].shape)

(1212,) (1212,)


In [25]:
impressions = pd.read_csv("../data/criteo/criteo_query_pool_impressions.csv")

In [43]:
columns = ["partner_id", "user_id", "product_category3", "count", "conversion_timestamp", "epsilon", "key"]
x = df.merge(impressions, how="left", on=["partner_id", "user_id"], suffixes=("", "impressions"))
synthetic_conversions = x.loc[x.click_timestamp.isna()][columns]
real_conversions = x.loc[~x.click_timestamp.isna()][columns]

synthetic_counts = synthetic_conversions.groupby(["partner_id", "key"]).user_id.count().to_dict()
real_counts = real_conversions.groupby(["partner_id", "key"]).user_id.count().to_dict()

In [44]:
completely_synthetic_queries = []
for key in synthetic_counts.keys():
    if key not in real_counts:
        completely_synthetic_queries.append(key)

completely_synthetic_queries

[('98594F28C91322F1DAF20DF86F3F6D8A', 13),
 ('A57EF8EBFE4E6EF71F0DC4877B186793', 30),
 ('A57EF8EBFE4E6EF71F0DC4877B186793', 32),
 ('A57EF8EBFE4E6EF71F0DC4877B186793', 34),
 ('F762E4949B07C39AEA4F2D26849B1893', 0),
 ('F762E4949B07C39AEA4F2D26849B1893', 2),
 ('F762E4949B07C39AEA4F2D26849B1893', 3),
 ('F762E4949B07C39AEA4F2D26849B1893', 4),
 ('F762E4949B07C39AEA4F2D26849B1893', 5)]